In [4]:
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import requests
from bs4 import BeautifulSoup
import csv

In [5]:
place_code = {"東京":{"prec_no":"44", "block_no":"47662", "type":"s1"}, "大分":{"prec_no":"83", "block_no":"0808", "type":"a1"}, "弘前":{"prec_no":"31", "block_no":"0166", "type":"a1"}}
base_url = "http://www.data.jma.go.jp/obd/stats/etrn/view/daily_%s.php?prec_no=%s&block_no=%s&year=%s&month=%s&day=&view=p1"

In [6]:
def str2float(str):
    try:
        return float(str)
    except:
        return 0.0

In [10]:
l = []
#都市を網羅します
for place in place_code.keys():
    #最終的にデータを集めるリスト (下に書いてある初期値は一行目。つまり、ヘッダー。)
    print(place)
    # index = place_name.index(place)
    prec_no = place_code[place]["prec_no"]
    block_no = place_code[place]["block_no"]
    data_type = place_code[place]["type"]

    year = 2021
    for month in range(1,2):
        #print(month)
        #2つの都市コードと年と月を当てはめる。
        r = requests.get(base_url%(data_type, prec_no, block_no, year, month))
        r.encoding = r.apparent_encoding

        # まずはサイトごとスクレイピング
        soup = BeautifulSoup(r.text)
        # findAllで条件に一致するものをすべて抜き出します。
        # 今回の条件はtrタグでclassがmtxになってるものです。
        rows = soup.findAll('tr',class_='mtx')

        # 表の最初の1~4行目はカラム情報なのでスライスする。(indexだから初めは0だよ)
        # 【追記】2020/3/11 申し訳ございません。間違えてました。
        if data_type == "a1":
            rows_data = rows[3:]

            # 1日〜最終日までの１行を網羅し、取得します。
            for row in rows_data:
                # 今度はtrのなかのtdをすべて抜き出します
                data = row.findAll('td')

                #１行の中には様々なデータがあるので全部取り出す。
                # ★ポイント
                day = data[0].findAll('a')[0].string # 日にち
                date = str(year) + "/" + str(month) + "/" + day
                precip_total  = data[1].string #降水量(mm)合計
                temp_ave = data[4].string #平均気温
                temp_high = data[5].string #最高気温
                temp_low = data[6].string #最低気温
                daylight = data[13].string #日照時間
                d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                l.append(d)
        else:
            rows_data = rows[4:]
            for row in rows_data:
                # 今度はtrのなかのtdをすべて抜き出します
                data = row.findAll('td')

                #１行の中には様々なデータがあるので全部取り出す。
                # ★ポイント
                day = data[0].findAll('a')[0].string
                date = str(year) + "/" + str(month) + "/" + day
                precip_total  = data[3].string #降水量(mm)合計
                temp_ave = data[6].string #平均気温
                temp_high = data[7].string #最高気温
                temp_low = data[8].string #最低気温
                daylight = data[16].string #日照時間
                d = {"date":date, "precip_total":precip_total, "temp_ave":temp_ave, "temp_high":temp_high, "temp_low":temp_low, "daylight":daylight, "place":place}
                l.append(d)

東京


IndexError: list index out of range

In [8]:
df = pd.DataFrame(l)

In [0]:
# Write recipe outputs
todays_weather = dataiku.Dataset("todays_weather")
todays_weather.write_with_schema(df)